pip install torch torchvision torchaudio
pip install opencv-python
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html

In [162]:
# Importar librerías necesarias
import fitz  # PyMuPDF
import re
import spacy
import json
from transformers import LayoutLMTokenizer, LayoutLMForTokenClassification

In [163]:
# Cargar modelo de spaCy
nlp = spacy.load("es_core_news_sm")

# Cargar modelo de LayoutLM
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased")

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Función para extraer texto del PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

In [165]:
def limpiar_texto(texto):
    """Normaliza y limpia el texto para mejorar la extracción de metadatos."""
    texto = re.sub(r'\s+', ' ', texto)  # Remueve espacios extra
    texto = re.sub(r'[^\x20-\x7E\xC0-\xFF]', '', texto)  # Remueve caracteres especiales
    return texto.strip()

def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto limpio."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text("text") + "\n"
    return limpiar_texto(texto)

In [166]:
def extract_metadata(text):
    metadata = {}

    metadata["titulo"] = "Gaceta del Congreso"

    # Entidades participantes
    entidades_match = re.search(r"(SENADO Y CÁMARA|SENADO|CÁMARA)", text, re.IGNORECASE)
    metadata["entidades"] = entidades_match.group(0) if entidades_match else "Desconocido"

    # ISSN
    issn_match = re.search(r"ISSN\s?(\d{4}-\d{4})", text)
    metadata["issn"] = issn_match.group(1) if issn_match else "No encontrado"

    # Año y número de publicación
    anio_num_match = re.search(r"AÑO\s+([XVI]+)\s+-\s+N°\s+(\d+)", text)
    metadata["anio"] = anio_num_match.group(1) if anio_num_match else "No encontrado"
    metadata["numero_publicacion"] = anio_num_match.group(2) if anio_num_match else "No encontrado"

    # Fecha
    fecha_match = re.search(r"(\w+,\s\d+\sde\s\w+\sde\s\d{4})", text)
    metadata["fecha"] = fecha_match.group(0) if fecha_match else "No encontrada"

    # Rama legislativa
    metadata["rama_legislativa"] = "RAMA LEGISLATIVA DEL PODER PÚBLICO"

    # Buscar la sección de directores -----------------------------
    # Extraer solo el nombre del director antes del título
    senado_match = re.search(r"\n([A-ZÁÉÍÓÚÑ\s]+)\nSECRETARIO\s+GENERAL\s+DEL\s+SENADO", text)
    camara_match = re.search(r"\n([A-ZÁÉÍÓÚÑ\s]+)\nSECRETARIO\s+GENERAL\s+DE\s+LA\s+CÁMARA", text)

    # Función para limpiar nombres (remueve espacios extra y caracteres erróneos)
    def clean_name(name):
        if name:
            name = name.strip()  # Eliminar espacios en los extremos
            name = re.sub(r'\s+', ' ', name)  # Reemplazar múltiples espacios por uno solo
            # Eliminar texto irrelevante si es necesario
            name = re.sub(r'^(REPÚBLICA|RAMA|SENADO|CÁMARA|DEL|PODER|PÚBLICO|LA|DE|COLOMBIA|RAMA|LEGISLATIVA|DEL|PODER|PÚBLICO)\s+', '', name, flags=re.IGNORECASE)
            return name
        return "No encontrado"

    metadata["directores"] = {
        "director_senado": clean_name(senado_match.group(1)) if senado_match else "No encontrado",
        "director_camara": clean_name(camara_match.group(1)) if camara_match else "No encontrado"
    }


    # Tipo de documento
    tipo_doc_match = re.search(r"(PONENCIAS|ACTAS|INFORMES|PROYECTOS DE LEY)", text)
    metadata["tipo_documento"] = tipo_doc_match.group(0) if tipo_doc_match else "No identificado"

    # Subtítulo
    subtitulo_match = re.search(r"([A-Z\s]+)\n+\"(.+?)\"", text)
    metadata["subtitulo"] = subtitulo_match.group(1).strip() if subtitulo_match else "No identificado"
    metadata["subsubtitulo"] = subtitulo_match.group(2).strip() if subtitulo_match else "No identificado"

    return metadata

In [167]:
# Función principal para extraer metadata desde PDF
def extract_metadata_from_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    metadata = extract_metadata(text)

    # Convertir a JSON
    metadata_json = json.dumps(metadata, indent=4, ensure_ascii=False)
    return metadata_json

In [168]:
pdf_path = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf"
metadata_json = extract_metadata_from_pdf(pdf_path)
print(metadata_json)

{
    "titulo": "Gaceta del Congreso",
    "entidades": "SENADO Y CÁMARA",
    "issn": "No encontrado",
    "anio": "No encontrado",
    "numero_publicacion": "No encontrado",
    "fecha": "Lunes, 28 de marzo de 2016",
    "rama_legislativa": "RAMA LEGISLATIVA DEL PODER PÚBLICO",
    "directores": {
        "director_senado": "GREGORIO ELJACH PACHECO",
        "director_camara": "DE COLOMBIA RAMA LEGISLATIVA DEL PODER PÚBLICO S E N A D O D E L A R E P Ú B L I C A JORGE HUMBERTO MANTILLA SERRANO"
    },
    "tipo_documento": "No identificado",
    "subtitulo": "No identificado",
    "subsubtitulo": "No identificado"
}


In [169]:
text = extract_text_from_pdf(pdf_path)
print(text) 

GACETA DEL CONGRESO  110  
Lunes, 28 de marzo de 2016 
Página 1
C O M E N TA R I O S
(Artículo 36,  Ley 5ª de 1992)
IMPRENTA   NACIONAL   DE   COLOMBIA
www.imprenta.gov.co
SENADO Y CÁMARA
AÑO XXV - Nº 110 
    Bogotá, D. C., lunes, 28 de marzo de 2016 
 EDICIÓN  DE  64  PÁGINAS
DIRECTORES: 
REPÚBLICA   DE   COLOMBIA
RAMA  LEGISLATIVA  DEL  PODER  PÚBLICO
S E N A D O   D E   L A   R E P Ú B L I C A
JORGE HUMBERTO MANTILLA SERRANO
SECRETARIO  GENERAL  DE  LA  CÁMARA
www.camara.gov.co 
GREGORIO ELJACH PACHECO
SECRETARIO  GENERAL  DEL  SENADO
www.secretariasenado.gov.co 
G A C E T A   D E L   C O N G R E S O
I S S N  0 1 2 3  -  9 0 6 6
COMENTARIOS AL PROYECTO DE LEY NÚMERO 97 DE 2015 SENADO
“por la cual se prohíbe la producción, comercialización, exportación, importación  
y distribución de cualquier variedad de asbesto en Colombia”. 
Mg
Mg
RESEÑA
respecto a la diferencia entre el crisotilo  
y los asbestos anﬁboles
“El asbesto” no es en si un mineral. Es un término colectivo que se da a 

In [170]:
# Imprimir la sección de directores para ver su estructura real
directores_match = re.search(r"DIRECTORES?:(.*?)\n(?:SECREATRIO\s+GENERAL\s+DE\s+LA\+NACION|www|SECRETARIO\s+GENERAL\s+DE\s+LA\s+CÁMARA|)", text, re.DOTALL)

print(directores_match.group(1) if directores_match else "No se encontró la sección de directores")
